In [3]:
import torch
import torchvision
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch import optim
from torch import nn
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
from torch.utils.data import Dataset
from torchinfo import summary

#from helper_functions import *
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (15.0, 7.0)

from helper_torch import *

In [32]:
class Conv_Bi_LSTM(nn.Module):
    def __init__(self, nfeat_x, nfeat_y, window_in, window_out):
        super().__init__()
        self.bilstm1 = nn.LSTM(input_size=nfeat_x, hidden_size=128, batch_first=True, bidirectional=True, dropout=0.5)
        self.conv1 = nn.Conv1d(in_channels=7, out_channels=7, kernel_size=64)
        self.bilstm2 = nn.LSTM(input_size=(256 -63), hidden_size=256, batch_first=True, bidirectional=True, dropout=0.5)
        self.conv2 = nn.Conv1d(in_channels=7, out_channels=7, kernel_size=128)
        self.bilstm3 = nn.LSTM(input_size=(256*2 - 127), hidden_size=128, batch_first=True, bidirectional=True, dropout=0.5)
        self.conv3=nn.Conv1d(in_channels=7, out_channels=7, kernel_size=64)
        self.relu = nn.ReLU()
        self.selu = nn.SELU()
        self.fc = nn.Linear(window_in * 193, nfeat_y * window_out)

        self.yfeat = nfeat_y
        self.xfeat = nfeat_x
        self.win_in = window_in
        self.win_out = window_out

    def forward(self, x):
        #print('x.shape: ', x.shape)
        x, _ = self.bilstm1(x)
        #print('bilstm1: ', x.shape)
        x = self.conv1(x)
        x = self.selu(x)
        #print('conv1 ',x.shape)
        x, _ = self.bilstm2(x)
        #print('bilstm2', x.shape)
        x = self.conv2(x)
        #print('conv2: ', x.shape)
        x = self.selu(x)
        x, _ = self.bilstm3(x)
        #print('bisltm3:', x.shape)
        x = self.conv3(x)
        x = self.relu(x)
        #print('conv3: ', x.shape)
        x = x.reshape((x.shape[0], -1))
        #print("x.reshape: ", x.shape)
        x = self.fc(x)
        #print('fc: ', x.shape)
        y = x.reshape((x.shape[0], self.win_out, self.yfeat))
        #print('final', y.shape)

        
        return y





In [33]:
model = Conv_Bi_LSTM(nfeat_x=4, nfeat_y=3, window_in=7, window_out=2)
summary(model, (1,7,4))


C:\Users\matha\anaconda3\envs\pytorch\lib\site-packages\torch\nn\modules\rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Layer (type:depth-idx)                   Output Shape              Param #
Conv_Bi_LSTM                             --                        --
├─LSTM: 1-1                              [1, 7, 256]               137,216
├─Conv1d: 1-2                            [1, 7, 193]               3,143
├─SELU: 1-3                              [1, 7, 193]               --
├─LSTM: 1-4                              [1, 7, 512]               923,648
├─Conv1d: 1-5                            [1, 7, 385]               6,279
├─SELU: 1-6                              [1, 7, 385]               --
├─LSTM: 1-7                              [1, 7, 256]               527,360
├─Conv1d: 1-8                            [1, 7, 193]               3,143
├─ReLU: 1-9                              [1, 7, 193]               --
├─Linear: 1-10                           [1, 6]                    8,112
Total params: 1,608,901
Trainable params: 1,608,901
Non-trainable params: 0
Total mult-adds (M): 14.76
Input size (MB): 0.00
For